In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
data=pd.read_csv('/content/data.csv')
data=data[['2','3','4']].rename(columns={'2':'current_1','3':'current_2','4':'current_3'})
data.head()

In [ ]:
data['current_1_r1000'] = data['current_1'].rolling(500).mean()
# data['MA336'] = data['current_2'].rolling(336).mean()
# plot 
import plotly.express as px
fig = px.line(data, x=data.index.values, y=['current_1', 'current_1_r1000'], title='test', template = 'plotly_dark')
fig.show()

In [ ]:
import numpy as np

# Define window size and sliding step
window_size = 500
step = 250

# Define statistical features to be calculated for each window
features = ['mean', 'std', 'min', 'max']

# Preprocess the data
X = []
for i in range(0, len(data), step): #-window_size from len(data)
    window = data.iloc[i:i+window_size]

    x = []
    for col in ['current_1', 'current_2', 'current_3']:
        for f in features:
            x.append(getattr(window[col], f)())
    X.append(x)
X = np.array(X)


In [ ]:
df=pd.DataFrame(X)
df.head()

In [ ]:
temp=pd.concate(data,X)

In [ ]:
! pip install --force-reinstall -v NumPy==1.19

In [ ]:
! pip install numba==0.53

In [ ]:
data.drop(['current_1_r10000'	,'current_1_r1000'],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
from pycaret.anomaly import *
s = setup(data, session_id = 123)

In [ ]:
models()

In [ ]:
# train model
iforest = create_model('iforest', fraction = 0.1)
iforest_results = assign_model(iforest)
iforest_results.head()

In [ ]:
iforest_results[iforest_results['Anomaly'] == 1].head()

In [ ]:
import plotly.graph_objects as go
# plot value on y-axis and date on x-axis
fig = px.line(iforest_results, x=iforest_results.index, y="current_1", title=' ANOMALY DETECTION', template = 'plotly_dark')
# create list of outlier_dates
outlier_dates = iforest_results[iforest_results['Anomaly'] == 1].index
# obtain y value of anomalies to plot
y_values = [iforest_results.loc[i]['current_1'] for i in outlier_dates]
fig.add_trace(go.Scatter(x=outlier_dates, y=y_values, mode = 'markers', 
                name = 'Anomaly', 
                marker=dict(color='red',size=10)))
        
fig.show()